# Tässä ohjelma, jolla voi muuntaa yritystukidatan verottajan kaipaamaan muotoon. Tämä tiedosto kannattaa kopioida omaan Drive- kansioon. 



# Aja koodia klikkaamalla vasemmassa sarakkeessa olevasta play-nappista. Solun alle ilmestyy viesti, josta voit ladata dataexcelit ohjelmaan. Data täytyy olla samassa muodossa kuin malliexcelissä, jotta ohjelma toimii. Tulostiedoston lataus käynnistyy lopuksi automaattisesti. Dataexceleitä voi valita useita, jolloin ohjelma poistaa duplikaatit joukosta.

# Verottajan dokumentaatio on myös jaetussa Google Drive-kansiossa.
https://drive.google.com/file/d/1Kxc6r1fJqvxVoxb5biEeCNvQtKhMJvPC/view?usp=sharing


# Ohjeet pähkinänkuoressa:

1. Kopioi koko VSJTUETE-kansio omaan Drive-kansioon.
2. Täydennä vsjtuete_template omilla tiedoillasi ja lataa se omalle koneelle.
  Pohja löytyy täältä: https://drive.google.com/file/d/1fezsBF6o5WU_tvic847DJQT96AEjw8X-/view?usp=sharing
3. Aja alla oleva koodi.



In [ ]:
import pandas as pd
import os
from datetime import datetime
import multiprocessing as mp
import numpy as np
from datetime import datetime
import random
from google.colab import files, drive
from IPython.display import Markdown, display, HTML
import io



def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))


# Määritellään tulostettavat rivit.

def apply_tax_notation(
                        tietuetunnus,
                       suorituslaji, 
                       maksajan_y_tunnus, 
                        
                       saajan_tunnus, 
                       tuki, 
                       maksupäivä, 
                        saajan_nimi,
                        aikaleima,
                       tuen_koodi,
                        suomeksi, 
                        ruotsiksi,
                        käyttötarkoitus,
                        energia,
                      ohjelmiston_tunnus,
                        
                        lopputunnus
                        ):

    
    
    return "000:"+tietuetunnus+"\n084:"+str(suorituslaji)+"\n010:"+str(maksajan_y_tunnus)+"\n083:"+str(saajan_tunnus)+"\n203:"+str(tuki)+"\n057:"+str(maksupäivä)+"\n085:"+str(saajan_nimi[:35])+"\n198:"+str(aikaleima)+"\n205:"+str(tuen_koodi)+"\n207:"+str(suomeksi)+"\n210:"+str(ruotsiksi)+"\n208:"+str(käyttötarkoitus)+"\n209:"+str(energia)+"\n014:"+str(ohjelmiston_tunnus)+"\n999:"+str(lopputunnus)+"\n"
        
# Ladataan tiedosto
uploaded = files.upload()

# Datan muokkaus


data = pd.concat([pd.read_excel(file) for file in list(uploaded.keys())]).drop_duplicates()


data['Suorituslaji']=data['Suorituslaji'].fillna(data.Suorituslaji.mode().values[0]).astype(int)
data['Tuen saajan henkilö-tai y-tunnus'] = data['Tuen saajan henkilö-tai y-tunnus'].str.strip().str.upper()


data['Maksettu tuki'] = data['Maksettu tuki'].astype(float).astype(str).str.replace('.',',')
data['Maksettu tuki'] = [c+'0' if len(c.split(',')[-1]) == 1 else c for c in data['Maksettu tuki']]
data['Tuen maksupäivä'] = [pd.to_datetime(c).strftime("%d%m%Y") for c in data['Tuen maksupäivä']]
data['Tuen maksupäivä'] = [c  if len(c)==8 else '0'+c for c in data['Tuen maksupäivä']]
data['Ohjelmiston tuottama aikaleima'] = (datetime.today()+pd.Timedelta(hours=2)).strftime("%d%m%Y%H%M%S")
data['Lopputunnus'] = np.arange(1, len(data)+1)
data = data.fillna('')


# Esimerkki osakeyhtiöistä. Data pitää olla tässä muodossa.
pretty_print(data[data['Tuen saajan nimi'].str.lower().str.contains('oy')].sample(10))


# Tulostettavan koostesarakkeen generointi

data['tax']=data.apply(lambda row: apply_tax_notation(
                                        row['Tietuetunnus'],
                                        row['Suorituslaji'],
                                         row['Tuen maksajan y-tunnus'],
                                         row['Tuen saajan henkilö-tai y-tunnus'],
                                         row['Maksettu tuki'],
                                         row['Tuen maksupäivä'],
                                         row['Tuen saajan nimi'],
                                        row['Ohjelmiston tuottama aikaleima'],
                                         row['Tuen koodi'],
                                        row['Tuen nimi suomeksi'],
                                        row['Tuen nimi ruotsiksi'],
                                        row['Tuen käyttötarkoitus'],
                                        row['Oikeus energiatuotteen valmisteveron palautukseen'],
                                         row['Ilmoituksen tuottaneen ohjelmiston yksilöivä tieto'],
                                          row['Lopputunnus']),
          axis=1)


#Kirjoitus tekstitiedostoon

filename = datetime.now().strftime('%Y%m%d_%H%M%S_')+data['Tietuetunnus'].values[0]+'_'+str(datetime.now().year-1)+'_'+str(random.randint(100,99999999))+'.txt'

for tax in data.tax:

    for line in tax.split('\n')[:-1]:

      with open(filename,'a') as f:
              f.write(line+'\r\n')
              
              
files.download(filename)